# LeNet

接下来的几天我们将主要精力用来重温以下经典。
![](./img/history.png)

卷积神经网络的鼻祖，Yan Lecun在1998年提出来的，卷积神经网络，其中最经典的当属LeNet5，这个网络其实一点也不简单，我们来实现一下，通过这个我们会对卷积结构有更深刻的理解

![](./img/lenet.jpg)

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchvision import transforms, datasets

# load mnist
def minst_loader():
    # load mnist
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])  

    train_set = datasets.MNIST('./data', train = True, download = True, transform = transform)
    test_set = datasets.MNIST('./data', train = False, download = True, transform = transform)

    train_loader = torch.utils.data.DataLoader(train_set, batch_size = 32, shuffle = True)
    test_loader = torch.utils.data.DataLoader(train_set, shuffle = False)
    return (train_loader, test_loader)


In [2]:
def train(data_loader, n_epoch, net, criterion):
    optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum = 0.5)

    for epoch in range(n_epoch):
        loss_arr = []
        accuracy_arr = []
        for i, data in enumerate(data_loader, 0): # 0的意思是无论重跑几次，都从0开始迭代，避免多次试验时不稳定的问题
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad() # 等同于net.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            _, predicted = torch.max(outputs, 1) # 注意一定是dim = 1，0是batch
            acc = (predicted == labels).long().sum()
            total = labels.size(0)

            loss_arr.append(loss.data[0])
            accuracy_arr.append(acc.data[0] * 100 / total)
          
        print("epoch: %d, Loss: %.2f, Accuracy: %.2f %%" % (epoch, np.sum(loss_arr) / len(loss_arr), np.sum(accuracy_arr) / len(accuracy_arr)))   

### 网络模型

- 将输入图像28x28x1转成32x32x1.

- 卷积层一：C1: 输出为 28x28x6. [ kernal_size = 5, feature_size = 6, stride = 1 ]
- 激活一：activation1: 自由选择.
- 池化层一S2：输出为 14 x 14 x 6.  [ 2 x 2 ] 尊重原著称为下采样层 downsampling


- 卷积层二：C3: 输出 10x10x16. [ kernal_size = 5, feature_size = 16, stride = 1 ]
- 激活二：activations2: 自由选择
- 池化层二S4：输出为5 x 5 x 16 尊重原著称为下采样层 downsampling S2

6是如何变成16的呢？这块会让初学者困惑，参见下图：
![](./img/lenets2c3.png)
6个feature map与S2层相连的3个feature map相连接。
6个feature map与S2层相连的4个feature map相连接。
3个feature map与S2层部分不相连的4个feature map相连接。
1个与S2层的所有feature map相连。

这里面的核心问题是，一个feature map如何与多个feature map相连呢？
![](./img/convconv.png)
feature之间类似于全连接关系，每个后置feature的一个点，对应之前多个feature的卷积后再加权复合。

对于LeNet5，S2与C3的连接参数计算方法是：每个卷积核大小是5x5，25个点位，三个feature并入一个，则3->6的参数量为：6 x (5 x 5 x 3 + 1)
同理，4->6：6 x (5 x 5 x 4 + 1)，4->3：3 x (5 x 5 x 4 + 1)，6->1：1 x (5 x 5 x 6 + 1)，共计1516个参数，输出16个大小为10 * 10的feature

- 维度变换：三维变一维， 输出为：
- 卷积层三C5： 输出为120  

#### 为什么叫C5呢？从图上看明明是全连接

由于S4的作用，输出为5x5x16所以，本层再使用5x5的卷积核时，发生了一个神奇的现象，只能卷积一下，形成一个1x1的图，使用了120个卷积核，因而有16->120的卷积映射关系，共有参数120 x (5x5x16 + 1) = 48120个参数。输出也就是120 x 1的类似全连接层的效果了，平展开就像一个全连接层。

- 全连接层一的激活：activation3

- 全连接层二F6： 输出为84
- 全连接层二的激活: activation4

- 全连接层三： 输出为10 (RBF，欧式径向基函数)


参考资料：
http://blog.csdn.net/zhangjunhit/article/details/53536915

In [ ]:
# 尝试使用Sequential
# 
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 6, 2), # c1
            nn.MaxPool2d()
        )